# Mandatory assignment 2: Solow growth model with human capital
Vision: Programming is more than writing code. The ultimate goal of the projects in this course
is that you learn to formulate a programming problem of your own choice, and find your own way
to solve it, and present the results. The bullets below are minimum requirements, but otherwise it is
very much up to you, what yo

* **Step 1: Package import:** The first part of code in our project is the use of libraries with essential packages of panda, numpy and glob which is respectively for data structuring and especially arrays and pathnames. On top of that we import statsmodel

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from scipy.stats import trim_mean, kurtosis
from scipy.stats.mstats import mode, gmean, hmean
from IPython.display import display

* **Step 2: Datapath and data processing:** Our data p

In [8]:
#Set datapath 
datapath = "Dataset/"
data = pd.read_excel(datapath + "APPENDIX_TableA_SECOND_ED-7.xls/")

#Remove unnecessary rows 
data=data[14:-18]
#Remove unnecessary columns 
data = data.iloc[:, :-3]

#Mark/select first row
new_header = data.iloc[0]
#all data below first row
data = data[1:] 
#use all data below first row and set column names equal new_header
data.columns = new_header

data.rename(columns = {list(data)[0]:"Country"}, inplace=True)
                                                    
data.head()

NotADirectoryError: [Errno 20] Not a directory: 'Dataset/APPENDIX_TableA_SECOND_ED-7.xls/'

In [2]:
%matplotlib inline
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import geopandas
import geoplot
import geoplot.crs as gcrs
import math
import sqlite3
import datetime as dt
import sqlalchemy
from numpy.random import random
from sqlalchemy import create_engine
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from pandas import DataFrame as df
from scipy.stats import trim_mean, kurtosis
from scipy.stats.mstats import mode, gmean, hmean

In [3]:
DATAPATH = "modelproject/Dataset/APPENDIX_TableA_SECOND_ED-7.xls/"

data_tennis = pd.concat(DATAPATH)





TypeError: first argument must be an iterable of pandas objects, you passed an object of type "str"